In [61]:
%load_ext autoreload
%autoreload 2

from scripts.wandb_experiments.gr_comparison import ComparisonExperiment
import multiprocessing as mp
import torch

torch.set_grad_enabled(False)
mp.set_start_method("spawn", force=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from scripts.wandb_runs.run_texgen import RunGenerativeRenderingConfig
from text3d2video.pipelines.generative_rendering_pipeline import (
    GenerativeRenderingConfig,
)
from text3d2video.pipelines.pipeline_utils import ModelConfig

decoder_paths = [
    "mid_block.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.2.transformer_blocks.0.attn1",
]

base_config = RunGenerativeRenderingConfig(
    prompt="Deadpool",
    animation_tag="booty:latest",
    generative_rendering=GenerativeRenderingConfig(
        module_paths=decoder_paths,
    ),
    model=ModelConfig(),
    seed=0,
)

gr_override = {
    "generative_rendering.do_pre_attn_injection": True,
    "generative_rendering.do_post_attn_injection": True,
}
controlnet_override = {
    "generative_rendering.do_pre_attn_injection": False,
    "generative_rendering.do_post_attn_injection": False,
}

anims = ["human_left_right:latest", "human_top_down:latest"]

e = ComparisonExperiment("prove_artefacts_stormtrooper")

ImportError: cannot import name 'RunGenerativeRenderingConfig' from 'scripts.wandb_runs.run_texgen' (/home/jorge/thesis/scripts/wandb_runs/run_texgen.py)

In [15]:
e.sync_experiment(rerun_all=False)


Would execute 1 new runs:
- exp

Would delete 0 outdated runs:

Do you want to continue? (y/n)
Deleting 0 runs
Executing 1 runs


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /home/jorge/thesis/wandb/run-20250415_180103-g08tam6a
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exp
wandb: ⭐️ View project at https://wandb.ai/romeu/diffusion-3D-features
wandb: 🚀 View run at https://wandb.ai/romeu/diffusion-3D-features/runs/g08tam6a
wandb:                                                                                
wandb: 
wandb: Run summary:
wandb: hash 14f4875fb31a4ef3bf54...
wandb: 
wandb: 🚀 View run exp at: https://wandb.ai/romeu/diffusion-3D-features/runs/g08tam6a
wandb: ⭐️ View project at: https://wandb.ai/romeu/diffusion-3D-features
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250415_180103

In [28]:
from omegaconf import OmegaConf


exp_run = [r for r in e.get_logged_runs() if r.name == "exp"][0]
exp_config = OmegaConf.create(exp_run.config)

exp_config.override_dims

[[{'animation_tag': 'human_left_right:latest'}, {'animation_tag': 'human_top_down:latest'}], [{'generative_rendering.do_pre_attn_injection': True, 'generative_rendering.do_post_attn_injection': True}, {'generative_rendering.do_pre_attn_injection': False, 'generative_rendering.do_post_attn_injection': False}]]

In [35]:
from omegaconf import OmegaConf
from scripts.wandb_experiments.gr_comparison import ComparisonExperimentConfig

cfg = ComparisonExperimentConfig(e.base_config, e.overrides)
cfg = OmegaConf.create(cfg)

In [47]:
for dim, overrides in enumerate(cfg.override_dims[0]):
    for k, v in overrides.items():
        print(k)

animation_tag
animation_tag


In [58]:
runs = e.get_runs_grouped()

AttributeError: 'str' object has no attribute 'items'

In [39]:
import numpy as np
from text3d2video.artifacts.video_artifact import VideoArtifact
from text3d2video.utilities.ipython_utils import display_vid
from text3d2video.utilities.video_comparison import video_grid
from text3d2video.utilities.video_util import pil_frames_to_clip
import wandb_util.wandb_util as wbu


def get_video(run):
    video = wbu.logged_artifacts(run)[0]
    video = VideoArtifact.from_wandb_artifact(video)
    return pil_frames_to_clip(video.read_frames(), 10)


videos = np.vectorize(get_video)(runs)

display_vid(video_grid(videos, x_labels=["GR", "ControlNet"]), width=1000)